# Training a Diffusion Model with Weights and Biases (W&B)

<!--- @wandbcode{dlai_02} -->

이 노트북에서는 W&B를 사용한 확산 모델의 학습을 계측해 보겠습니다. "Difussion 모델의 작동 방식"(https://www.deeplearning.ai/short-courses/how-diffusion-models-work/) 코스 노트북을 확장하겠습니다.

- 트레이닝 손실 및 메트릭 로깅
- 훈련 중 모델에서 샘플링하고 샘플을 W&B에 업로드하기
- 모델 체크포인트를 W&B에 저장하기

In [1]:
# !pip install wandb

In [2]:
from types import SimpleNamespace
from pathlib import Path
from tqdm.notebook import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from utilities import *

import wandb

We encourage you to create an account to get the full user experience from W&B

In [3]:
wandb.login(anonymous="allow")

wandb: Currently logged in as: kimwooglae. Use `wandb login --relogin` to force relogin


True

## Setting Things Up

In [4]:
# we are storing the parameters to be logged to wandb
DATA_DIR = Path('./data/')
SAVE_DIR = Path('./data/weights/')
SAVE_DIR.mkdir(exist_ok=True, parents=True)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

config = SimpleNamespace(
    # hyperparameters
    num_samples = 30,

    # diffusion hyperparameters
    timesteps = 500,
    beta1 = 1e-4,
    beta2 = 0.02,

    # network hyperparameters
    n_feat = 64, # 64 hidden dimension feature
    n_cfeat = 5, # context vector is of size 5
    height = 16, # 16x16 image
    
    # training hyperparameters
    batch_size = 100,
    n_epoch = 32,
    lrate = 1e-3,
)

### DDPM 노이즈 스케줄러 및 샘플러를 설정합니다(Diffusion Course와 동일).
- perturb_input: 스케줄의 해당 시간 간격에 입력 이미지에 노이즈를 추가합니다.
- sample_ddpm_context: DDPM 샘플러를 사용하여 이미지를 생성합니다. 훈련 중에 이 함수를 사용하여 모델에서 정기적으로 샘플링하고 훈련이 어떻게 진행되고 있는지 확인합니다.

In [5]:
# setup ddpm sampler functions
perturb_input, sample_ddpm_context = setup_ddpm(config.beta1, 
                                                config.beta2, 
                                                config.timesteps, 
                                                DEVICE)

In [6]:
# construct model
nn_model = ContextUnet(in_channels=3, 
                       n_feat=config.n_feat, 
                       n_cfeat=config.n_cfeat, 
                       height=config.height).to(DEVICE)

In [7]:
# load dataset and construct optimizer
dataset = CustomDataset.from_np(path=DATA_DIR)
dataloader = DataLoader(dataset, 
                        batch_size=config.batch_size, 
                        shuffle=True)
optim = torch.optim.Adam(nn_model.parameters(), lr=config.lrate)

## Training

확산을 안내하기 위해 각 클래스의 샘플 6개가 포함된 고정 컨텍스트 벡터를 선택합니다.

In [8]:
# Noise vector
# x_T ~ N(0, 1), sample initial noise
noises = torch.randn(config.num_samples, 3, 
                     config.height, config.height).to(DEVICE)  

# A fixed context vector to sample from
ctx_vector = F.one_hot(torch.tensor([0,0,0,0,0,0,   # hero
                                     1,1,1,1,1,1,   # non-hero
                                     2,2,2,2,2,2,   # food
                                     3,3,3,3,3,3,   # spell
                                     4,4,4,4,4,4]), # side-facing 
                       5).to(DEVICE).float()

다음 트레이닝 셀은 CPU에서 실행하는 데 매우 오래 걸리며, 이미 GPU가 장착된 머신에서 모델을 트레이닝했습니다.

###  [트레이닝 결과](https://wandb.ai/dlai-course/dlai_sprite_diffusion/runs/pzs3gsyo)를 확인하실 수 있습니다.

In [ ]:
# create a wandb run
run = wandb.init(project="dlai_sprite_diffusion", 
                 job_type="train", 
                 config=config)

# we pass the config back from W&B
config = wandb.config

for ep in tqdm(range(config.n_epoch), leave=True, total=config.n_epoch):
    # set into train mode
    nn_model.train()
    optim.param_groups[0]['lr'] = config.lrate*(1-ep/config.n_epoch)
    
    pbar = tqdm(dataloader, leave=False)
    for x, c in pbar:   # x: images  c: context
        optim.zero_grad()
        x = x.to(DEVICE)
        c = c.to(DEVICE)   
        context_mask = torch.bernoulli(torch.zeros(c.shape[0]) + 0.8).to(DEVICE)
        c = c * context_mask.unsqueeze(-1)        
        noise = torch.randn_like(x)
        t = torch.randint(1, config.timesteps + 1, (x.shape[0],)).to(DEVICE) 
        x_pert = perturb_input(x, t, noise)      
        pred_noise = nn_model(x_pert, t / config.timesteps, c=c)      
        loss = F.mse_loss(pred_noise, noise)
        loss.backward()    
        optim.step()

        wandb.log({"loss": loss.item(),
                   "lr": optim.param_groups[0]['lr'],
                   "epoch": ep})


    # save model periodically
    if ep%4==0 or ep == int(config.n_epoch-1):
        nn_model.eval()
        ckpt_file = SAVE_DIR/f"context_model.pth"
        torch.save(nn_model.state_dict(), ckpt_file)

        artifact_name = f"{wandb.run.id}_context_model"
        at = wandb.Artifact(artifact_name, type="model")
        at.add_file(ckpt_file)
        wandb.log_artifact(at, aliases=[f"epoch_{ep}"])

        samples, _ = sample_ddpm_context(nn_model, 
                                         noises, 
                                         ctx_vector[:config.num_samples])
        wandb.log({
            "train_samples": [
                wandb.Image(img) for img in samples.split(1)
            ]})
        
# finish W&B run
wandb.finish()

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

...

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/894 [00:00<?, ?it/s]